In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
import torch

In [34]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\reg_TA_PA_DL_new.csv"
df = pd.read_csv(file)

In [35]:
#Create the dummy variables, one for each country
df_dummies = pd.get_dummies(df['admin1'])
df_with_dummies = df.join(df_dummies)

#Create the dummy variables, one for each month
df['month'] = pd.DatetimeIndex(df['time']).month_name()
df_dummies_m = pd.get_dummies(df['month'])
#
df_with_dummies = df_with_dummies.join(df_dummies_m)
df_with_dummies['month'] = pd.DatetimeIndex(df['time']).month

#Create the dummy variables, one for each for each country-month pair
df_dummies_mr = pd.get_dummies(df['admin1'] + df['month'])
df_with_dummies = df_with_dummies.join(df_dummies_mr)
df_with_dummies = df_with_dummies.replace({True: 1, False: 0})

In [36]:
#add a column with the TA with noise
df_with_dummies['TA_noise'] = df_with_dummies['TA'] + np.random.normal(0, 0.1, len(df_with_dummies))
df_with_dummies['PA_noise'] = df_with_dummies['PA'] + np.random.normal(0, 0.1, len(df_with_dummies))
df_with_dummies['DL_noise'] = df_with_dummies['DL'] + np.random.normal(0, 0.1, len(df_with_dummies))

In [37]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [46]:
unit_names = df['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df['month'].unique().tolist()
unit_names_mr = (df['admin1'] + df['month']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
for dummy_name_mr in unit_names_mr[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + Awdal + Bakool + Banadir + Bari + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Nugaal + Sanaag + Sool + Togdheer + March + April + May + June + July + August + September + October + November + December + January + AwdalMarch + AwdalApril + AwdalMay + AwdalJune + AwdalJuly + AwdalAugust + AwdalSeptember + AwdalOctober + AwdalNovember + AwdalDecember + AwdalJanuary + AwdalFebruary + BakoolMarch + BakoolApril + BakoolMay + BakoolJune + BakoolJuly + BakoolAugust + BakoolSeptember + BakoolOctober + BakoolNovember + BakoolDecember + BakoolJanuary + BakoolFebruary + BanadirMarch + BanadirApril + BanadirMay + BanadirJune + BanadirJuly + BanadirAugust + BanadirSeptember + BanadirOctober + BanadirNovember + BanadirDecember + BanadirJanuary + BanadirFebruary + BariMarch + BariApril + BariMay + BariJune + BariJuly + BariAugust + BariSeptember + BariOctober + BariNovember

In [47]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     16.11
Date:                Wed, 19 Jul 2023   Prob (F-statistic):               0.00
Time:                        12:33:56   Log-Likelihood:                -18933.
No. Observations:                5148   AIC:                         3.830e+04
Df Residuals:                    4929   BIC:                         3.974e+04
Df Model:                         218                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [40]:
#with open('ols_summary_2.txt', 'w') as f:
 #   f.write(lsdv_model_results.summary().as_text())

In [48]:
lsdv_model = smf.ols(formula=form, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     131.0
Date:                Wed, 19 Jul 2023   Prob (F-statistic):           8.84e-82
Time:                        12:34:34   Log-Likelihood:                -20129.
No. Observations:                5148   AIC:                         4.027e+04
Df Residuals:                    5144   BIC:                         4.029e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.6585      0.204     17.975      0.0